<div style="text-align: center"><h1 style="text-decoration: underline;">DSML Project</h1></div>



This is the official Notebook of the DSML Project from Marc Rennefort, Kilian Lipinsky, Timo Hagelberg, Jan Behrendt-Emden and Paul Severin. In order to create this Project we used the following dataset: https://data.cityofchicago.org/Transportation/Transportation-Network-Providers-Trips-2023-2024-/n26f-ihde/about_data
<h4>1. Description</h4>
The goal of this Project is to predict the tip for ride-hailing in Chicago based on the travelled time and distance as well as the money spent, the weather and the information if the customer matched to a shared trip or not. 


In [1]:
#Note all your imports here


import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LinearRegression
from datetime import datetime
from meteostat import Hourly, Point


In [ ]:
# Gesamten Datensatz laden
# Bei wenig RAM: Direkt übernächste nächste Zelle ausführen

data = pd.read_csv('Data/Chicago_RideHailing_Data.csv')  #habt geduld, dauert beim laden

In [ ]:
# Basic Infos anzeigen
# Kann übersprungen werden 
data.head()
data.info()
data.isnull().sum()
# nicht benötigte Spalten entfernen
data_cleaned = data.drop(columns = ['Percent Time Chicago', 'Percent Distance Chicago', 'Pickup Census Tract', 'Dropoff Census Tract',
                            'Pickup Community Area', 'Dropoff Community Area', 'Additional Charges', 'Trips Pooled', 
                            'Pickup Centroid Latitude', 'Pickup Centroid Location', 'Dropoff Centroid Latitude', 'Dropoff Centroid Longitude', 'Dropoff Centroid Location', 'Pickup Centroid Longitude'])
data_cleaned.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27423899 entries, 0 to 27423898
Data columns (total 24 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Trip ID                     object 
 1   Trip Start Timestamp        object 
 2   Trip End Timestamp          object 
 3   Trip Seconds                float64
 4   Trip Miles                  float64
 5   Percent Time Chicago        float64
 6   Percent Distance Chicago    float64
 7   Pickup Census Tract         float64
 8   Dropoff Census Tract        float64
 9   Pickup Community Area       float64
 10  Dropoff Community Area      float64
 11  Fare                        float64
 12  Tip                         float64
 13  Additional Charges          float64
 14  Trip Total                  float64
 15  Shared Trip Authorized      object 
 16  Shared Trip Match           object 
 17  Trips Pooled                float64
 18  Pickup Centroid Latitude    float64
 19  Pickup Centroid Lon

,Trip ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Fare,Tip,Trip Total,Shared Trip Authorized,Shared Trip Match
0,8255c0b48b877bda14fbc2f490b6e42f359c44ec,09/19/2023 06:15:00 PM,09/19/2023 07:15:00 PM,3109.0,23.5,32.5,0.0,40.70,False,False
1,8255c1529cdd828c3ce4e6bdc494c734e8432025,09/28/2023 12:15:00 PM,09/28/2023 12:45:00 PM,1640.0,14.7,30.0,0.0,38.74,False,False
2,8255c247359d6e148930ba860afa3282848c67ed,09/03/2023 05:15:00 PM,09/03/2023 05:30:00 PM,772.0,1.9,20.0,0.0,22.39,False,False
3,8255c37a57997b77f75da6af598a38fc28b4e832,09/13/2023 03:30:00 PM,09/13/2023 03:30:00 PM,452.0,1.1,10.0,0.0,14.07,False,False
4,8255c6b273a32b3a76d7dad03328abb229de7fbd,09/16/2023 01:00:00 PM,09/16/2023 01:00:00 PM,819.0,3.9,10.0,0.0,12.89,False,False


In [ ]:
# Direkt nur die benötigten Spalten laden (mein Laptop hat nicht genug RAM für den gesamten Datensatz haha)
# Die vorherige Zelle muss nicht ausgeführt werden!

data_cleaned = pd.read_csv('Data/Chicago_RideHailing_Data.csv', usecols= ['Trip ID', 'Trip Start Timestamp', 'Trip End Timestamp', 'Trip Seconds', 'Trip Miles', 'Fare', 'Tip', 'Trip Total', 'Shared Trip Authorized', 'Shared Trip Match'])
data_cleaned.head()

In [6]:
#drop all rows with null values
data_cleaned = data_cleaned.dropna(axis = 0)

In [9]:
#changing format of the date (Trip Start Timestamp)
data_cleaned['Trip Start Timestamp'] = pd.to_datetime(data_cleaned['Trip Start Timestamp'],  format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
data_cleaned['Trip End Timestamp'] = pd.to_datetime(data_cleaned['Trip End Timestamp'],  format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

In [10]:
print('Null-Werte: ', data_cleaned.isnull().sum())
data_cleaned.info()
data_cleaned.head()

Null-Werte:  Trip ID                   0
Trip Start Timestamp      0
Trip End Timestamp        0
Trip Seconds              0
Trip Miles                0
Fare                      0
Tip                       0
Trip Total                0
Shared Trip Authorized    0
Shared Trip Match         0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 27354156 entries, 0 to 27423897
Data columns (total 10 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   Trip ID                 object        
 1   Trip Start Timestamp    datetime64[ns]
 2   Trip End Timestamp      datetime64[ns]
 3   Trip Seconds            float64       
 4   Trip Miles              float64       
 5   Fare                    float64       
 6   Tip                     float64       
 7   Trip Total              float64       
 8   Shared Trip Authorized  object        
 9   Shared Trip Match       object        
dtypes: datetime64[ns](2), float64(5), object(3)
memory 

,Trip ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Fare,Tip,Trip Total,Shared Trip Authorized,Shared Trip Match
0,8255c0b48b877bda14fbc2f490b6e42f359c44ec,2023-09-19 18:15:00,2023-09-19 19:15:00,3109.0,23.5,32.5,0.0,40.70,False,False
1,8255c1529cdd828c3ce4e6bdc494c734e8432025,2023-09-28 12:15:00,2023-09-28 12:45:00,1640.0,14.7,30.0,0.0,38.74,False,False
2,8255c247359d6e148930ba860afa3282848c67ed,2023-09-03 17:15:00,2023-09-03 17:30:00,772.0,1.9,20.0,0.0,22.39,False,False
3,8255c37a57997b77f75da6af598a38fc28b4e832,2023-09-13 15:30:00,2023-09-13 15:30:00,452.0,1.1,10.0,0.0,14.07,False,False
4,8255c6b273a32b3a76d7dad03328abb229de7fbd,2023-09-16 13:00:00,2023-09-16 13:00:00,819.0,3.9,10.0,0.0,12.89,False,False


In [11]:
# Wetterdaten laden:
wetter = Hourly(Point(41.8781, -87.6298), datetime(2023, 1, 1), datetime(2024, 12, 31))
wetter = wetter.fetch()
wetter.head()

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
time,,,,,,,,,,,
2023-01-01 00:00:00,2.0,-1.3,79.0,0.0,NaN,121.0,6.0,NaN,1013.0,NaN,3.0
2023-01-01 01:00:00,2.2,-1.1,79.0,0.0,NaN,140.0,5.4,NaN,1013.8,NaN,3.0
2023-01-01 02:00:00,2.8,-1.0,76.0,0.0,NaN,140.0,14.8,NaN,1013.2,NaN,3.0
2023-01-01 03:00:00,2.8,0.0,82.0,0.0,NaN,0.0,0.0,NaN,1012.9,NaN,3.0
2023-01-01 04:00:00,2.8,0.0,82.0,0.0,NaN,0.0,0.0,NaN,1012.9,NaN,12.0


In [12]:
# Wetterdaten cleanen:
wetter_cleaned = wetter.drop(columns=['dwpt', 'rhum', 'wdir', 'pres'])
wetter_cleaned = wetter_cleaned.rename(columns={'time': 'Time', 'temp': 'Temperature in C', 'prcp': 'Rain in mm', 'snow': 'Snow in mm', 'wspd': 'Wind Speed in km/h', 'wpgt': 'Peak Wind Speed in km/h', 'tsun': 'Sunshine Duration in min', 'coco': 'Weather Condition Code'})
wetter_cleaned.head()

,Temperature in C,Rain in mm,Snow in mm,Wind Speed in km/h,Peak Wind Speed in km/h,Sunshine Duration in min,Weather Condition Code
time,,,,,,,
2023-01-01 00:00:00,2.0,0.0,NaN,6.0,NaN,NaN,3.0
2023-01-01 01:00:00,2.2,0.0,NaN,5.4,NaN,NaN,3.0
2023-01-01 02:00:00,2.8,0.0,NaN,14.8,NaN,NaN,3.0
2023-01-01 03:00:00,2.8,0.0,NaN,0.0,NaN,NaN,3.0
2023-01-01 04:00:00,2.8,0.0,NaN,0.0,NaN,NaN,12.0


#### Erklärung der Weather Condition Codes

| Code | Weather Condition |
|------|-------------------|
| 1    | Clear             |
| 2    | Fair              |
| 3    | Cloudy            |
| 4    | Overcast          |
| 5    | Fog               |
| 6    | Freezing Fog      |
| 7    | Light Rain        |
| 8    | Rain              |
| 9    | Heavy Rain        |
| 10   | Freezing Rain     |
| 11   | Heavy Freezing Rain |
| 12   | Sleet             |
| 13   | Heavy Sleet       |
| 14   | Light Snowfall    |
| 15   | Snowfall          |
| 16   | Heavy Snowfall    |
| 17   | Rain Shower       |
| 18   | Heavy Rain Shower |
| 19   | Sleet Shower      |
| 20   | Heavy Sleet Shower |
| 21   | Snow Shower       |
| 22   | Heavy Snow Shower |
| 23   | Lightning         |
| 24   | Hail              |
| 25   | Thunderstorm      |
| 26   | Heavy Thunderstorm |
| 27   | Storm             |

Quelle: https://dev.meteostat.net/formats.html#weather-condition-codes